# InstaMark: Embedding and Extracting Watermarks using Discrete Wavelet Transform

<center>

![Image](images/instamark.png)

</center>
This notebook demonstrates how to embed and extract a watermark using the Discrete Wavelet Transform (DWT) method. It provides a practical application of DWT for steganography.

### Author Information

- **Name**: Sergio Mancini
- **Surname**: Mancini
- **ID**: 1000070116
- **Course**: Multimedia
- **A.Y.**: 2024/2025

## What is DWT (Discrete Wavelet Transform)?

The Discrete Wavelet Transform (DWT) is a mathematical technique used for transforming a signal or image into a set of wavelet coefficients. Unlike other transforms such as the Fourier Transform, DWT provides both frequency and spatial information, making it ideal for applications like image compression and steganography.

In steganography, DWT is used to embed secret data (like a watermark) into the wavelet coefficients of an image. This method is preferred because it provides a good balance between imperceptibility (the hidden watermark is not easily visible) and robustness (the watermark can withstand some image transformations, such as compression or noise).

DWT decomposes an image into different frequency subbands, including approximation and detail coefficients. By embedding the watermark into the high-frequency components, we can ensure that the hidden information does not affect the image's visual appearance significantly.


In [8]:
!pip install PyWavelets 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import cv2
import pywt
import numpy as np
import matplotlib.pyplot as plt

def dwt_embed(input_image, watermark_image, seed=2024):
    if len(input_image.shape) > 2 or len(watermark_image.shape) > 2:
        print("Parameter input_image and watermark_image should be grayscale")
        return input_image

    # Apply DWT to the input image
    cA_l1, (cH_l1, cV_l1, cD_l1) = pywt.dwt2(input_image.astype(np.float32), 'haar')
    cA_l2, (cH_l2, cV_l2, cD_l2) = pywt.dwt2(cA_l1, 'haar')

    # Resize watermark to fit the subband size
    wm_h, wm_w = cH_l2.shape
    watermark_image = cv2.resize(watermark_image, (wm_w, wm_h)).astype(np.float32)

    alpha = 2  
    cH_l2_modified = cH_l2 + alpha * watermark_image

    # Reconstruct the image with modified watermark
    cA_l1_reconstructed = pywt.idwt2((cA_l2, (cH_l2_modified, cV_l2, cD_l2)), 'haar')

    # Ensure the sizes match before applying inverse DWT
    cH_l1 = cv2.resize(cH_l1, (cA_l1_reconstructed.shape[1], cA_l1_reconstructed.shape[0])).astype(np.float32)
    cV_l1 = cv2.resize(cV_l1, (cA_l1_reconstructed.shape[1], cA_l1_reconstructed.shape[0])).astype(np.float32)
    cD_l1 = cv2.resize(cD_l1, (cA_l1_reconstructed.shape[1], cA_l1_reconstructed.shape[0])).astype(np.float32)

    marked_image = pywt.idwt2((cA_l1_reconstructed, (cH_l1, cV_l1, cD_l1)), 'haar')

    return np.clip(marked_image, 0, 255).astype(np.uint8)


def dwt_extract(marked_image, original_watermark, seed=2024):
    if len(marked_image.shape) > 2:
        print("Parameter marked_image should be grayscale")
        return marked_image

    # DWT to extract watermark
    cA_l1, (cH_l1, cV_l1, cD_l1) = pywt.dwt2(marked_image.astype(np.float32), 'haar')
    cA_l2, (cH_l2, cV_l2, cD_l2) = pywt.dwt2(cA_l1, 'haar')

    # Resize watermark to the correct dimensions
    height, width = marked_image.shape
    original_watermark = cv2.resize(original_watermark, (width >> 2, height >> 2))
    original_watermark = original_watermark.astype(np.float32)

    alpha = 3
    extracted_watermark = cH_l2 * original_watermark + cV_l2 * original_watermark
    extracted_watermark = 255 * extracted_watermark / np.max(extracted_watermark)
    extracted_watermark[extracted_watermark < alpha] = 0
    extracted_watermark[extracted_watermark >= alpha] = 255

    return extracted_watermark.astype(np.uint8)


if __name__ == '__main__':
    img_gray = cv2.imread('images/dexter.png', cv2.IMREAD_GRAYSCALE)
    img_watermark = cv2.imread('images/logo2.png', cv2.IMREAD_GRAYSCALE)
    _, img_watermark = cv2.threshold(img_watermark, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    img_marked = dwt_embed(img_gray, img_watermark, 20240211)
    cv2.imwrite('images/cover_marked.png', img_marked)

    img_stego = cv2.imread('images/cover_marked.png', cv2.IMREAD_GRAYSCALE)
    cover_marked = cv2.GaussianBlur(img_stego, (5, 5), 0)
    cover_marked_rotated = cv2.rotate(img_stego, cv2.ROTATE_180)

    temp_jpeg_path = 'images/temp_compressed_image.jpg'
    jpeg_compression_level = 95
    cv2.imwrite(temp_jpeg_path, img_stego, [cv2.IMWRITE_JPEG_QUALITY, jpeg_compression_level])
    compressed_img = cv2.imread(temp_jpeg_path, cv2.IMREAD_GRAYSCALE)
    
    img_watermark_extracted = dwt_extract(img_stego, img_watermark, 20240211)
    img_watermark_extracted2 = dwt_extract(cover_marked, img_watermark, 20240211)
    img_watermark_extracted3 = dwt_extract(cover_marked_rotated, img_watermark, 20240211)
    img_watermark_extracted4 = dwt_extract(compressed_img, img_watermark, 20240211)

    
    plt.figure(figsize=(12, 9))

    
    plt.subplot(231), plt.imshow(img_gray, cmap='gray'), plt.title('Cover'), plt.axis('off')
    plt.subplot(232), plt.imshow(img_marked, cmap='gray'), plt.title('Cover con Watermark'), plt.axis('off')
    plt.subplot(233), plt.imshow(img_watermark, cmap='gray'), plt.title('Watermark'), plt.axis('off')
    plt.subplot(234), plt.imshow(img_watermark_extracted, cmap='gray'), plt.title('Watermark Estratto'), plt.axis('off')
    plt.subplot(235), plt.imshow(img_watermark_extracted2, cmap='gray'), plt.title('Watermark Estratto BLUR'), plt.axis('off')
    plt.subplot(236), plt.imshow(img_watermark_extracted4, cmap='gray'), plt.title('Watermark Estratto COMPRESSIONE'), plt.axis('off')

    plt.tight_layout()
    plt.show()


## Code Explanation
### DWT Embed Function
1. **Input Validation:** The function first checks if both the input image and watermark image are grayscale.

2. **DWT Decomposition:** The input image is decomposed into its approximation and detail coefficients using the dwt2 function from the pywt library (Haar wavelet in this case).

3. **Watermark Resizing:** The watermark image is resized to fit the size of the detail subband cH_l2.

4. **Watermark Embedding:** The watermark is embedded in the cH_l2 subband, with the embedding strength controlled by the alpha parameter.

5. **Reconstruction:** The modified coefficients are used to reconstruct the image using the inverse DWT (idwt2).
Output: The final marked image is returned, clipped to ensure pixel values stay within the 0-255 range.

### DWT Extract Function
1. **Input Validation:** The function checks if the marked image is grayscale.

2. **DWT Decomposition:** The dwt2 function is applied to decompose the marked image into approximation and detail coefficients.

3. **Watermark Extraction:** The watermark is extracted from the high-frequency subbands (cH_l2 and cV_l2), using a threshold value (alpha) to distinguish between watermark and noise.

4. **Normalization:** The extracted watermark is normalized and thresholded to produce a binary image.

### Main Function
In the main function, the process of embedding and extracting the watermark is demonstrated step by step:

1. **Image Loading:** The cover image and watermark are loaded and thresholded.

2. **Watermark Embedding:** The dwt_embed function is used to embed the watermark into the cover image.

3. **Image Manipulation:** The marked image is subjected to various transformations (blurring, rotation, and compression).

4. **Watermark Extraction:** The dwt_extract function is applied to extract the watermark from the transformed images.

5. **Visualization:** The results are visualized using matplotlib to display the original, marked, and extracted watermarks.

## Conclusions
This notebook demonstrates the use of Discrete Wavelet Transform (DWT) for watermark embedding and extraction. The method provides a robust way to hide a watermark in the high-frequency components of an image, making it resistant to common image transformations such as blurring, and compression.

The watermark is embedded into the wavelet coefficients of the image, which allows for imperceptible changes to the cover image.

This technique can be extended to support other types of wavelet transforms and more advanced watermarking schemes, making it a powerful tool for digital watermarking in multimedia applications.